In [242]:
import pandas as pd

In [243]:
df = pd.read_csv("SemArt/semart_train.csv", sep="	", encoding="unicode_escape")
df.columns = df.columns.str.lower()

In [244]:
len(df["timeframe"].unique())

22

In [245]:
df.columns

Index(['image_file', 'description', 'author', 'title', 'technique', 'date',
       'type', 'school', 'timeframe'],
      dtype='object')

In [246]:
df.groupby("timeframe").size()

timeframe
0751-0800       2
0801-0850       5
0851-0900       3
0951-1000       4
1001-1050       1
1051-1100      26
1101-1150      14
1151-1200      21
1201-1250      29
1251-1300     116
1301-1350     670
1351-1400     347
1401-1450     938
1451-1500    2059
1501-1550    2576
1551-1600    1582
1601-1650    3404
1651-1700    2088
1701-1750    1444
1751-1800    1114
1801-1850    1056
1851-1900    1745
dtype: int64

In [247]:
df.groupby("school").size().sort_values()

school
Finnish             5
Portuguese          5
Polish             16
Norwegian          18
Irish              21
Other              22
Swedish            23
Bohemian           29
Catalan            35
Greek              37
Belgian            45
Scottish           46
Russian            68
Swiss              89
Danish             94
American          113
Hungarian         178
Austrian          216
Netherlandish     217
English           411
Spanish           852
German           1173
Flemish          2016
French           2556
Dutch            2948
Italian          8011
dtype: int64

In [248]:
df.groupby("type").size().sort_values()

type
study             57
interior         452
other            509
historical       545
still-life       926
genre           1630
mythological    1862
landscape       2490
portrait        3292
religious       7481
dtype: int64

In [249]:
technique_df = df.groupby("technique").size().to_frame()

# Development
## Setup
Setting up configurations such as label encoding and different types of data sets

In [250]:
from enum import Enum


class DataSetEnum(Enum):
    TRAIN = "train"
    TEST = "test"
    VALIDATION = "val"


class OutputEnum(Enum):
    TIMEFRAME = "timeframe"
    SCHOOL = "school"
    TYPE = "type"


timeframe_encoder = {
    "0801-0850": 0,
    "0851-0900": 0,
    "0751-0800": 0,
    "0951-1000": 0,
    "1001-1050": 0,
    "1051-1100": 0,
    "1101-1150": 0,
    "1151-1200": 0,
    "1201-1250": 0,
    "1251-1300": 0,
    "1301-1350": 0,
    "1351-1400": 0,
    "1401-1450": 1,
    "1451-1500": 1,
    "1501-1550": 2,
    "1551-1600": 2,
    "1601-1650": 3,
    "1651-1700": 3,
    "1701-1750": 4,
    "1751-1800": 4,
    "1801-1850": 5,
    "1851-1900": 5,
}

school_encoder = {
    "Finnish": 0,
    "Portuguese": 0,
    "Polish": 0,
    "Norwegian": 0,
    "Irish": 0,
    "Other": 0,
    "Swedish": 0,
    "Bohemian": 0,
    "Catalan": 0,
    "Greek": 0,
    "Belgian": 0,
    "Scottish": 0,
    "Russian": 0,
    "Swiss": 0,
    "Danish": 0,
    "American": 0,
    "Hungarian": 0,
    "Austrian": 0,
    "Netherlandish": 0,
    "English": 0,
    "Spanish": 1,
    "German": 2,
    "Flemish": 3,
    "French": 4,
    "Dutch": 5,
    "Italian": 6,
}

type_encoder = {
    "study": 0,
    "interior": 0,
    "other": 0,
    "historical": 0,
    "still-life": 0,
    "genre": 1,
    "mythological": 2,
    "landscape": 3,
    "portrait": 4,
    "religious": 5,
}

## CNN
Building the model, we are using a pre-trained CNN model called resnet.

In [251]:
import torch
import torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


class CNN(nn.Module):
    def __init__(self, output_dim: int):
        super(CNN, self).__init__()
        self.output_dim = output_dim
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
        self.resnet.requires_grad_(False)
        self.resnet.fc = nn.Linear(2048, output_dim)
        self.resnet.fc.requires_grad_(True)

    def forward(self, x):
        if len(x.shape) == 3:
            x = x.unsqueeze(0)
        x = self.resnet(x)
        return x


Using device: cuda


## Dataloader

We make use of a data loader that dynamically loads the dataset so that we avoid loading all of the images (training set has over 19k images) into memory. We also define some transformations, that are following how resnet was trained. For the training set we are doing some data augmentation to avoid overfitting while training.

In [252]:
from pathlib import Path
import numpy as np
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms
from PIL import Image


class ImageDataset(Dataset):
    def __init__(
        self,
        data_set: DataSetEnum,
        output: OutputEnum,
        encoder: dict[str, int] | None = None,
    ):
        self.data_set = data_set
        self.output = output
        self.data = pd.read_csv(
            f"SemArt/semart_{data_set.value}.csv", sep="	", encoding="unicode_escape"
        )
        self.data.columns = self.data.columns.str.lower()
        self._base_path = Path() / "SemArt" / "Images"
        self.transform = transforms.Compose(
            [
                # Get the values from here: https://pytorch.org/vision/0.18/models/generated/torchvision.models.resnet50.html#torchvision.models.resnet50
                transforms.Resize(232),
                # Data augmentation, randomly crop the image and flip it horizontally
                # only for the training set
                transforms.RandomCrop(224)
                if data_set == DataSetEnum.TRAIN
                else transforms.CenterCrop(224),
                transforms.RandomHorizontalFlip()
                if data_set == DataSetEnum.TRAIN
                else transforms.Lambda(lambda x: x),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        )
        if not encoder:
            print("Using standard encoder")
            self.encoder = (
                timeframe_encoder
                if output == OutputEnum.TIMEFRAME
                else school_encoder
                if output == OutputEnum.SCHOOL
                else type_encoder
                if output == OutputEnum.TYPE
                else None
            )
        else:
            self.encoder = encoder
        assert self.encoder is not None, "Invalid output type"
        self.data = self.data[self.data[self.output.value].isin(self.encoder.keys())]
        self.data = self.data.reset_index(drop=True)
        self._output_dim = len(set(self.encoder.values()))

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, idx: int) -> tuple[torch.Tensor, int]:
        # Set label
        label = -1
        label = self.encoder[self.data[self.output.value][idx]]  # type: ignore

        assert label >= 0, "No label found, invalid output type"

        # load image
        raw_image = Image.open(
            str((self._base_path / str(self.data["image_file"][idx])).resolve())
        ).convert("RGB")
        image: torch.Tensor = self.transform(raw_image)  # type: ignore
        image = image.to(device)
        return image, label


### Oversampling
Because the dataset is imbalanced in the classes that we are training on we have to use oversampling, here we create a `WeightedRandomSampler` that create probabilities based on how common a class is, this is only used for the training set.

In [253]:
def get_weights_for_balanced_sampler(dataset: ImageDataset) -> np.ndarray:
    output_dim = dataset._output_dim
    class_sizes = np.zeros(output_dim)
    encoder = dataset.encoder
    assert encoder is not None
    for key, val in encoder.items():
        class_sizes[val] += len(dataset.data[dataset.data[dataset.output.value] == key])

    weights = np.zeros(len(dataset))
    for idx, row in enumerate(dataset.data.iterrows()):
        weights[idx] = len(dataset) / class_sizes[encoder[row[1][dataset.output.value]]]

    return weights


### Training

In [254]:
from torch.utils.tensorboard.writer import SummaryWriter

# Use tensorboard to visualize the training process
writer = SummaryWriter()

In [255]:
import torch.nn.functional as F
import matplotlib.pyplot as plt


# Helper function to plot the class prediction
def plot_class_prediction(net, images, labels):
    preds = net(images)
    probs = F.softmax(preds, dim=1)
    fig = plt.figure(figsize=(6, 6))
    labels = labels.cpu().numpy()
    images = images.cpu()
    probs = probs.cpu().detach().numpy()
    for idx in np.arange(6):
        ax = fig.add_subplot(2, 3, idx + 1, xticks=[], yticks=[])
        image = images[idx].numpy().transpose(1, 2, 0)
        image = (image - image.min()) / (image.max() - image.min())
        ax.imshow(image)
        ax.set_title(
            f"{labels[idx]} -> {probs[idx].argmax()}({probs[idx].max():.2f})",
            color=("green" if probs[idx].argmax() == labels[idx] else "red"),
        )
    return fig

In [256]:
from IPython.display import clear_output
from tqdm.notebook import tqdm


# TRAIN
def train(model, train_dataloader, criterion, optimizer, writer, epoch):
    size = len(train_dataloader)
    model.train()
    try:
        for batch, (images, labels) in tqdm(
            enumerate(train_dataloader),
            total=size,
            leave=False,
            desc=f"Training {epoch + 1}",
            position=1,
        ):
            pred = model(images)
            labels = labels.to(device)
            loss = criterion(pred, labels)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            if batch % 10 == 0:
                # accuracy
                accuracy = (pred.argmax(1) == labels).sum().item() / len(labels)
                writer.add_scalars(
                    "Train/Loss", {"train": loss.item()}, batch + size * epoch
                )
                writer.add_scalars(
                    "Train/Accuracy", {"train": accuracy}, batch + size * epoch
                )
                # print(
                #     f"{batch}/{size} - train loss: {loss.item():.4f} - train accuracy: {accuracy:.4f}"
                # )

    except KeyboardInterrupt:
        print("Training interrupted")
        torch.save(model.state_dict(), "model.pth")
        raise KeyboardInterrupt


# TEST
def test(model, test_dataloader, criterion, writer, epoch):
    size = len(test_dataloader)
    model.eval()
    with torch.no_grad():
        confusion_matrix = np.zeros(
            (test_dataloader.dataset._output_dim, test_dataloader.dataset._output_dim)
        )
        random_index = np.random.randint(0, size)
        for batch, (images, labels) in tqdm(
            enumerate(test_dataloader),
            total=size,
            leave=False,
            desc=f"Testing {epoch + 1}",
            position=1,
        ):
            pred = model(images)
            labels = labels.to(device)
            loss = criterion(pred, labels)
            accuracy = (pred.argmax(1) == labels).sum().item() / len(labels)
            writer.add_scalars("Test/Loss", {"test": loss.item()}, batch + size * epoch)
            writer.add_scalars(
                "Test/Accuracy", {"test": accuracy}, batch + size * epoch
            )
            # print(
            #     f"{batch}/{size} - test loss: {loss.item():.4f} - test accuracy: {accuracy:.4f}"
            # )

            # confusion matrix
            for idx, (p, l) in enumerate(zip(pred.argmax(1), labels)):
                confusion_matrix[l.item(), p.item()] += 1

            if batch == random_index:
                writer.add_figure(
                    "Class Prediction",
                    plot_class_prediction(model, images, labels),
                    epoch,
                    close=True,
                )

        # plot confusion matrix
        fig = plt.figure()
        plt.imshow(confusion_matrix, cmap="viridis")
        plt.colorbar()
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        writer.add_figure("Confusion Matrix", fig, epoch, close=True)


### Validation
We calculate some metrics on the validation set and show some examples

In [257]:
def validate(model, validation_dataset: DataLoader, criterion):
    # Validation
    model.eval()
    test_dataloader = validation_dataset

    running_loss = 0.0
    running_accuracy = 0.0
    with torch.no_grad():
        for batch_n, (images, labels) in enumerate(
            tqdm(test_dataloader, total=len(test_dataloader), leave=False)
        ):
            pred = model(images)
            labels = labels.to(device)
            loss = criterion(pred, labels)
            running_loss += loss.item()
            running_accuracy += (pred.argmax(1) == labels).sum().item() / len(labels)

    print(f"Test loss: {running_loss / len(test_dataloader):.3f}")
    print(f"Test accuracy: {running_accuracy / len(test_dataloader):.3f}")

    return (
        running_accuracy / len(test_dataloader),
        running_loss / len(test_dataloader),
    )

In [258]:
criterion = nn.CrossEntropyLoss()
criterion.to(device)

CrossEntropyLoss()

In [259]:
def train_and_validation(model_class, encoder: dict[str, int], output: OutputEnum):
    train_dataset = ImageDataset(DataSetEnum.TRAIN, output, encoder)
    weights = get_weights_for_balanced_sampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=64,
        sampler=WeightedRandomSampler(weights, len(weights)),
    )
    test_dataset = ImageDataset(DataSetEnum.TEST, output, encoder)
    test_dataloader = DataLoader(test_dataset, batch_size=64)

    model = model_class(train_dataset._output_dim)
    model.to(device)
    params = model.parameters()
    # Optimizer parameters is same as Adam default in tensorflow 
    optimizer = torch.optim.Adam(params, eps=1e-07)

    epochs = 10
    for epoch in tqdm(
        range(epochs), desc="Epochs", leave=True, total=epochs, position=0
    ):
        train(model, train_dataloader, criterion, optimizer, writer, epoch)
        test(model, test_dataloader, criterion, writer, epoch)

    validation_dataset = ImageDataset(
        DataSetEnum.VALIDATION, output, encoder
    )
    validation_dataloader = DataLoader(validation_dataset, batch_size=64)
    accuracy, loss = validate(
        model, validation_dataloader, criterion
    )

    torch.save(
        model.state_dict(),
        f"model_trained_{output.value}_{test_dataset._output_dim}.pth",
    )
    return accuracy, loss

In [260]:
# School encodings
school_encoding_one = {
    "American": 0,
    "Austrian": 1,
    "Belgian": 2,
    "Bohemian": 3,
    "Danish": 4,
    "Dutch": 5,
    "English": 6,
    "Flemish": 7,
    "French": 8,
    "German": 9,
    "Hungarian": 10,
    "Irish": 11,
    "Italian": 12,
    "Netherlandish": 13,
    "Other": 14,
    "Polish": 15,
    "Portuguese": 16,
    "Russian": 17,
    "Scottish": 18,
    "Spanish": 19,
    "Swedish": 20,
    "Swiss": 21,
    "Catalan": 22,
    "Finnish": 23,
    "Norwegian": 24,
    "Greek": 25,
}

school_encoding_two = {
    "American": 0,
    "Austrian": 4,
    "Belgian": 4,
    "Bohemian": 3,
    "Danish": 1,
    "Dutch": 4,
    "English": 4,
    "Flemish": 4,
    "French": 4,
    "German": 4,
    "Hungarian": 3,
    "Irish": 4,
    "Italian": 2,
    "Netherlandish": 4,
    "Other": 5,
    "Polish": 3,
    "Portuguese": 2,
    "Russian": 3,
    "Scottish": 4,
    "Spanish": 2,
    "Swedish": 1,
    "Swiss": 4,
    "Catalan": 2,
    "Finnish": 1,
    "Norwegian": 1,
    "Greek": 2,
}

type_encoding = {
    "genre": 0,
    "historical": 1,
    "interior": 2,
    "landscape": 3,
    "mythological": 4,
    "other": 5,
    "portrait": 6,
    "religious": 7,
    "still-life": 8,
    "study": 9,
}

timeframe_encoding_one = {
    "0801-0850": 0,
    "0851-0900": 0,
    "0751-0800": 0,
    "0951-1000": 0,
    "1001-1050": 0,
    "1051-1100": 0,
    "1101-1150": 0,
    "1151-1200": 0,
    "1201-1250": 0,
    "1251-1300": 0,
    "1301-1350": 0,
    "1351-1400": 0,
    "1401-1450": 1,
    "1451-1500": 2,
    "1501-1550": 3,
    "1551-1600": 4,
    "1601-1650": 5,
    "1651-1700": 6,
    "1701-1750": 7,
    "1751-1800": 8,
    "1801-1850": 9,
    "1851-1900": 10,
}

timeframe_encoding_two = {
    "0801-0850": 0,
    "0851-0900": 0,
    "0751-0800": 0,
    "0951-1000": 0,
    "1001-1050": 0,
    "1051-1100": 0,
    "1101-1150": 0,
    "1151-1200": 0,
    "1201-1250": 0,
    "1251-1300": 0,
    "1301-1350": 0,
    "1351-1400": 0,
    "1401-1450": 1,
    "1451-1500": 1,
    "1501-1550": 2,
    "1551-1600": 2,
    "1601-1650": 3,
    "1651-1700": 3,
    "1701-1750": 4,
    "1751-1800": 4,
    "1801-1850": 5,
    "1851-1900": 5,
}

In [ ]:
results = pd.DataFrame(columns=["Accuracy", "Loss"])

acc, loss = train_and_validation(CNN, school_encoding_one, OutputEnum.SCHOOL)
results.loc["School Non-grouped"] = [acc, loss]

acc, loss = train_and_validation(CNN, school_encoding_two, OutputEnum.SCHOOL)
results.loc["School Grouped"] = [acc, loss]

acc, loss = train_and_validation(CNN, type_encoding, OutputEnum.TYPE)
results.loc["Type Encoding"] = [acc, loss]

acc, loss = train_and_validation(CNN, timeframe_encoding_one, OutputEnum.TIMEFRAME)
results.loc["Timeframe Non-grouped"] = [acc, loss]

acc, loss = train_and_validation(CNN, timeframe_encoding_two, OutputEnum.TIMEFRAME)
results.loc["Timeframe Grouped"] = [acc, loss]


Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Training 1:   0%|          | 0/301 [00:00<?, ?it/s]

In [ ]:
results

In [ ]:

results.to_csv("results.csv")